## Линеарна и полиномијална регресија за одређивање цене бундева - Лекција 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Инфографика: Дасани Мадипали</figcaption>


#### Увод

До сада сте истраживали шта је регресија користећи пример података из скупа података о ценама бундева који ћемо користити током ове лекције. Такође сте је визуализовали помоћу `ggplot2`. 💪

Сада сте спремни да дубље уђете у регресију за машинско учење. У овој лекцији ћете научити више о две врсте регресије: *основна линеарна регресија* и *полиномијална регресија*, као и нешто од математике која стоји иза ових техника.

> Кроз овај курикулум претпостављамо минимално знање математике и настојимо да га учинимо доступним студентима из других области, па обратите пажњу на напомене, 🧮 дијаграме и друге алате за учење који ће вам помоћи у разумевању.

#### Припрема

Подсећамо, учитавате ове податке како бисте постављали питања о њима.

-   Када је најбоље време за куповину бундева?

-   Коју цену могу да очекујем за кутију минијатурних бундева?

-   Да ли да их купим у корпама од пола бушела или у кутијама од 1 1/9 бушела? Хајде да наставимо са истраживањем ових података.

У претходној лекцији, креирали сте `tibble` (модерно преобликовање оквира података) и попунили га делом оригиналног скупа података, стандардизујући цене по бушелу. Међутим, на тај начин сте успели да сакупите само око 400 података и то само за јесење месеце. Можда можемо добити мало више детаља о природи података ако их боље очистимо? Видећемо... 🕵️‍♀️

За овај задатак биће нам потребни следећи пакети:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) је [збирка R пакета](https://www.tidyverse.org/packages) дизајнирана да учини науку о подацима бржом, лакшом и забавнијом!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) је оквир који представља [збирку пакета](https://www.tidymodels.org/packages/) за моделирање и машинско учење.

-   `janitor`: [janitor пакет](https://github.com/sfirke/janitor) пружа једноставне алате за испитивање и чишћење "прљавих" података.

-   `corrplot`: [corrplot пакет](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) пружа визуелни алат за истраживање корелационе матрице који подржава аутоматско ређање променљивих како би се открили скривени обрасци међу променљивима.

Можете их инсталирати на следећи начин:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Скрипта испод проверава да ли имате потребне пакете за завршетак овог модула и инсталира их за вас у случају да недостају.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Учитаћемо ове сјајне пакете и учинити их доступним у нашој тренутној R сесији. (Ово је само за илустрацију, `pacman::p_load()` је то већ урадио за вас)

## 1. Линија линеарне регресије

Као што сте научили у Лекцији 1, циљ вежбе линеарне регресије је да се нацрта *линија* *најбољег* *прилагођавања* како би се:

-   **Приказали односи између променљивих**. Приказао однос између променљивих.

-   **Направиле предикције**. Направиле тачне предикције о томе где би нова тачка података могла пасти у односу на ту линију.

Да бисмо нацртали ову врсту линије, користимо статистичку технику која се зове **Регресија најмањих квадрата**. Термин `најмањи квадрати` значи да су све тачке података које окружују регресиону линију подигнуте на квадрат и затим сабране. Идеално, та коначна сума је што је могуће мања, јер желимо низак број грешака, односно `најмање квадрате`. Као таква, линија најбољег прилагођавања је линија која нам даје најнижу вредност за збир квадрата грешака - отуда назив *регресија најмањих квадрата*.

Ово радимо јер желимо да моделирамо линију која има најмању кумулативну удаљеност од свих наших тачака података. Такође подижемо термине на квадрат пре него што их саберемо, јер нас занима њихова величина, а не правац.

> **🧮 Покажите ми математику**
>
> Ова линија, названа *линија најбољег прилагођавања*, може се изразити [једначином](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` је '`објашњавајућа променљива` или `предиктор`'. `Y` је '`зависна променљива` или `исход`'. Нагиб линије је `b`, а `a` је пресек са Y-осом, што се односи на вредност `Y` када је `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "нагиб = $y/x$")
    Инфографика: Џен Лупер
>
> Прво, израчунајте нагиб `b`.
>
> Другим речима, и позивајући се на оригинално питање о подацима о бундевама: "предвидите цену бундеве по бушелу по месецу", `X` би се односио на цену, а `Y` на месец продаје.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Инфографика: Џен Лупер
> 
> Израчунајте вредност Y. Ако плаћате око 4 долара, мора да је април!
>
> Математика која израчунава линију мора показати нагиб линије, који такође зависи од пресека, односно где се `Y` налази када је `X = 0`.
>
> Можете видети метод израчунавања ових вредности на веб сајту [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Такође посетите [овај калкулатор најмањих квадрата](https://www.mathsisfun.com/data/least-squares-calculator.html) да бисте видели како вредности бројева утичу на линију.

Није тако страшно, зар не? 🤓

#### Корелација

Још један термин који треба разумети је **Коефицијент корелације** између датих X и Y променљивих. Користећи дијаграм расејања, можете брзо визуализовати овај коефицијент. Дијаграм са тачкама података распоређеним у уредну линију има високу корелацију, али дијаграм са тачкама података распршеним свуда између X и Y има ниску корелацију.

Добар модел линеарне регресије биће онај који има висок (ближи 1 него 0) Коефицијент корелације користећи метод Регресије најмањих квадрата са линијом регресије.


## **2. Плес са подацима: креирање дата фрејма који ће се користити за моделирање**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Илустрација: @allison_horst</figcaption>


<!--![Илустрација: \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Учитајте потребне библиотеке и скуп података. Претворите податке у оквир података који садржи подскуп података:

-   Узмите само тикве чија је цена одређена по бушелу

-   Претворите датум у месец

-   Израчунајте цену као просек високих и ниских цена

-   Претворите цену тако да одражава цену по количини бушела

> Ове кораке смо обрадили у [претходној лекцији](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

У духу чисте авантуре, хајде да истражимо [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) који пружа једноставне функције за испитивање и чишћење неуређених података. На пример, хајде да погледамо називе колона за наше податке:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Можемо боље. Хајде да направимо ова имена колона `friendR` тако што ћемо их претворити у [snake_case](https://en.wikipedia.org/wiki/Snake_case) конвенцију користећи `janitor::clean_names`. Да бисте сазнали више о овој функцији: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Толико tidyR 🧹! Сада, плес са подацима користећи `dplyr`, као у претходној лекцији! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Одличан посао!👌 Сада имате чист, уредан скуп података на којем можете изградити свој нови регресиони модел!

Шта кажете на расејани графикон?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Дијаграм распршивања нас подсећа да имамо податке о месецима само од августа до децембра. Вероватно нам је потребно више података како бисмо могли да извучемо закључке на линеаран начин.

Хајде да поново погледамо наше податке за моделирање:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Шта ако бисмо желели да предвидимо `price` тикве на основу колона `city` или `package`, које су типа карактер? Или још једноставније, како бисмо могли да пронађемо корелацију (која захтева да оба уноса буду нумеричка) између, рецимо, `package` и `price`? 🤷🤷

Модели машинског учења најбоље функционишу са нумеричким карактеристикама уместо текстуалних вредности, тако да је генерално потребно претворити категоријалне карактеристике у нумеричке репрезентације.

То значи да морамо пронаћи начин да преобликујемо наше предикторе како бисмо их учинили погоднијим за ефикасно коришћење у моделу, процес познат као `feature engineering`.


## 3. Предобрада података за моделирање са рецептурама 👩‍🍳👨‍🍳

Активности које преобликују вредности предиктора како би их учиниле лакшим за ефикасно коришћење модела називају се `инжењеринг карактеристика`.

Различити модели имају различите захтеве за предобраду. На пример, метода најмањих квадрата захтева `кодирање категоријалних променљивих` као што су месец, сорта и назив града. Ово једноставно подразумева `превод` колоне са `категоријалним вредностима` у једну или више `нумеричких колона` које замењују оригиналну.

На пример, претпоставимо да ваши подаци укључују следећу категоријалну карактеристику:

|  град   |
|:-------:|
| Денвер  |
| Најроби |
|  Токио  |

Можете применити *ордирално кодирање* да замените јединствену целобројну вредност за сваку категорију, овако:

| град |
|:----:|
|  0   |
|  1   |
|  2   |

И то је оно што ћемо урадити са нашим подацима!

У овом одељку, истражићемо још један невероватан пакет из Tidymodels-а: [recipes](https://tidymodels.github.io/recipes/) - који је осмишљен да вам помогне да предобрадите своје податке **пре** тренинга модела. У својој суштини, рецепт је објекат који дефинише које кораке треба применити на скуп података како би био спреман за моделирање.

Сада, хајде да направимо рецепт који припрема наше податке за моделирање заменом јединственог целобројног броја за све опсервације у колонама предиктора:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Сјајно! 👏 Управо смо направили наш први рецепт који одређује исход (цену) и његове одговарајуће предикторе, а такође и да све колоне предиктора треба да буду кодиране у скуп целих бројева 🙌! Хајде да брзо разложимо шта смо урадили:

-   Позив функције `recipe()` са формулом говори рецепту *улоге* променљивих користећи податке `new_pumpkins` као референцу. На пример, колона `price` је добила улогу `outcome`, док су остале колоне добиле улогу `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` одређује да сви предиктори треба да буду претворени у скуп целих бројева, при чему нумерација почиње од 0.

Сигурни смо да вам можда падају на памет мисли попут: "Ово је тако кул!! Али шта ако треба да проверим да ли рецепти раде тачно оно што очекујем? 🤔"

То је сјајна мисао! Видите, када једном дефинишете рецепт, можете проценити параметре који су потребни за стварну претобраду података, а затим извући обрађене податке. Обично ово није потребно када користите Tidymodels (ускоро ћемо видети уобичајену конвенцију -\> `workflows`), али може бити корисно када желите да урадите неку врсту провере како бисте потврдили да рецепти раде оно што очекујете.

За то ће вам бити потребна још два глагола: `prep()` и `bake()`, а као и увек, наши мали пријатељи из R-а од стране [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) помоћи ће вам да ово боље разумете!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Илустрација од @allison_horst</figcaption>


<!--![Илустрација од \@allison_horst](../../../../../../2-Regression/3-Linear/images/recipes.png){width="550"}-->


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): процењује потребне параметре из тренинг скупа који касније могу бити примењени на друге скупове података. На пример, за дату колону предиктора, који посматрач ће бити додељен као цео број 0, 1, 2 итд.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): узима припремљен рецепт и примењује операције на било који скуп података.

С тим речено, хајде да припремимо и применимо наше рецепте како бисмо заиста потврдили да ће, у позадини, колоне предиктора прво бити кодиране пре него што се модел прилагоди.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Ура! 🥳 Обрађени подаци `baked_pumpkins` имају све своје предикторе кодиране, што потврђује да ће кораци предобраде дефинисани као наш рецепт функционисати како је очекивано. Ово чини податке теже читљивим за вас, али много разумљивијим за Tidymodels! Одвојите мало времена да откријете која је опсервација мапирана на одговарајући целобројни вредност.

Такође је вредно напоменути да је `baked_pumpkins` оквир података на којем можемо изводити прорачуне.

На пример, хајде да покушамо да пронађемо добру корелацију између две тачке ваших података како бисмо потенцијално изградили добар предиктивни модел. Користићемо функцију `cor()` да то урадимо. Укуцајте `?cor()` да бисте сазнали више о функцији.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Испоставило се да постоји само слаба корелација између Града и Цене. Међутим, постоји нешто боља корелација између Пакета и његове Цене. То има смисла, зар не? Обично, што је већа кутија са производима, то је виша цена.

Док смо већ код тога, хајде да покушамо и да визуализујемо матрицу корелације свих колона користећи пакет `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Много боље.

Добро питање које сада можемо поставити о овим подацима је: '`Коју цену могу очекивати за одређени пакет бундева?`' Хајде да одмах пређемо на то!

> Напомена: Када **`bake()`** припремљени рецепт **`pumpkins_prep`** са **`new_data = NULL`**, добијате обрађене (тј. кодиране) податке за обуку. Ако имате други скуп података, на пример тест сет, и желите да видите како би рецепт обрадио те податке, једноставно бисте испекли **`pumpkins_prep`** са **`new_data = test_set`**.

## 4. Изградња модела линеарне регресије

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Инфографика: Дасани Мадипали</figcaption>


Сада када смо направили рецепт и заправо потврдили да ће подаци бити одговарајуће претходно обрађени, хајде сада да направимо регресиони модел како бисмо одговорили на питање: `Коју цену могу очекивати за одређени пакет бундеве?`

#### Обучавање линеарног регресионог модела користећи тренинг сет

Као што сте вероватно већ закључили, колона *price* је `излазна` променљива, док је колона *package* `предикторска` променљива.

Да бисмо то урадили, прво ћемо поделити податке тако да 80% иде у тренинг сет, а 20% у тест сет, затим ћемо дефинисати рецепт који ће кодирати предикторску колону у скуп целих бројева, а потом ћемо направити спецификацију модела. Нећемо припремати и обрађивати наш рецепт јер већ знамо да ће он претходно обрадити податке како је очекивано.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Одлично! Сада када имамо рецепт и спецификацију модела, потребно је да пронађемо начин да их спојимо у један објекат који ће прво обрадити податке (припрема + обрада у позадини), затим обучити модел на обрађеним подацима, а такође омогућити потенцијалне активности пост-обраде. Како ти се то чини за мир у души!🤩

У оквиру Tidymodels-а, овај практични објекат се зове [`workflow`](https://workflows.tidymodels.org/) и згодно чува све компоненте твог модела! Ово је оно што бисмо у *Python*-у назвали *pipelines*.

Хајде да све спакујемо у један workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Узгред, ток рада може бити прилагођен/трениран на сличан начин као што се тренира модел.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Из излаза модела можемо видети коефицијенте који су научени током тренинга. Они представљају коефицијенте праве најбољег прилагођавања која нам даје најмању укупну грешку између стварне и предвиђене променљиве.

#### Процена перформанси модела коришћењем тест скупа

Време је да видимо како се модел показао 📏! Како то радимо?

Сада када смо обучили модел, можемо га користити за прављење предвиђања за test_set користећи `parsnip::predict()`. Затим можемо упоредити ова предвиђања са стварним вредностима ознака како бисмо проценили колико добро (или не!) модел функционише.

Хајде да почнемо са прављењем предвиђања за тест скуп, а затим да повежемо колоне са тест скупом.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Да, управо сте обучили модел и користили га за прављење предикција! 🔮 Да ли је добар? Хајде да проценимо перформансе модела!

У оквиру Tidymodels-а, ово радимо помоћу `yardstick::metrics()`! За линеарну регресију, фокусираћемо се на следеће метрике:

-   `Root Mean Square Error (RMSE)`: Квадратни корен од [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Ова метрика даје апсолутну вредност у истој јединици као и ознака (у овом случају, цена бундеве). Што је вредност мања, то је модел бољи (у поједностављеном смислу, представља просечну цену за коју су предикције погрешне!).

-   `Coefficient of Determination (обично познат као R-squared или R2)`: Релативна метрика код које је већа вредност боља за модел. У суштини, ова метрика представља колико варијансе између предвиђених и стварних вредности ознака модел може да објасни.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Ту иде перформанс модела. Хајде да видимо да ли можемо добити бољу индикацију визуелизацијом расејаног графикона пакета и цене, а затим користити предвиђања за преклапање линије најбољег уклапања.

То значи да ћемо морати припремити и обрадити тестни сет како бисмо кодирали колону пакета, а затим је повезати са предвиђањима која је наш модел направио.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Одлично! Као што можете видети, линеарни регресиони модел не успева добро да генерализује однос између пакета и његове одговарајуће цене.

🎃 Честитамо, управо сте направили модел који може помоћи у предвиђању цене неколико врста бундева. Ваш празнични бундева врт ће бити прелеп. Али вероватно можете направити бољи модел!

## 5. Направите полиномијални регресиони модел

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Инфографик од Дасани Мадипали</figcaption>


<!--![Инфографик од Дасани Мадипали](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Понекад наши подаци можда немају линеарну везу, али и даље желимо да предвидимо исход. Полиномијална регресија може нам помоћи да направимо предвиђања за сложеније нелинеарне односе.

Узмимо, на пример, однос између паковања и цене у нашем скупу података о бундевама. Иако понекад постоји линеарна веза између променљивих - што је већа бундева по запремини, то је већа цена - понекад се ти односи не могу приказати као раван или права линија.

> ✅ Ево [још неких примера](https://online.stat.psu.edu/stat501/lesson/9/9.8) података који би могли користити полиномијалну регресију
>
> Поново погледајте однос између сорте и цене на претходном графикону. Да ли овај распршени графикон изгледа као да би нужно требало да се анализира правом линијом? Можда не. У овом случају, можете пробати полиномијалну регресију.
>
> ✅ Полиноми су математички изрази који могу садржати једну или више променљивих и коефицијената

#### Тренирајте модел полиномијалне регресије користећи скуп за тренирање

Полиномијална регресија креира *закривљену линију* како би боље одговарала нелинеарним подацима.

Хајде да видимо да ли ће полиномијални модел боље функционисати у прављењу предвиђања. Следићемо донекле сличан поступак као што смо радили раније:

-   Направите рецепт који одређује кораке предобраде који треба да се спроведу на нашим подацима како би били спремни за моделирање, тј. кодирање предиктора и израчунавање полинома степена *n*

-   Направите спецификацију модела

-   Спојите рецепт и спецификацију модела у радни ток

-   Креирајте модел тако што ћете прилагодити радни ток

-   Процените колико добро модел функционише на тест подацима

Хајде да почнемо!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Оцена перформанси модела

👏👏 Направили сте полиномни модел, хајде да направимо предвиђања на тест скупу!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Ву-ху, хајде да проценимо како је модел извршио на test_set користећи `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Много бољи учинак.

`rmse` се смањио са отприлике 7 на отприлике 3, што указује на смањење грешке између стварне цене и предвиђене цене. Ово можете *слободно* тумачити као да су у просеку нетачне прогнозе погрешне за око \$3. `rsq` се повећао са отприлике 0.4 на 0.8.

Сви ови показатељи указују на то да полиномски модел ради много боље од линеарног модела. Одличан посао!

Хајде да видимо да ли можемо ово да визуализујемо!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Можете видети закривљену линију која боље одговара вашим подацима! 🤩

Можете је учинити још глаткијом тако што ћете проследити полиномску формулу функцији `geom_smooth` на следећи начин:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Баш као глатка крива!🤩

Ево како можете направити нову прогнозу:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Предвиђање помоћу `полиномског модела` има смисла, с обзиром на распршене графике `цена` и `пакета`! И, ако је ово бољи модел од претходног, гледајући исте податке, потребно је да планирате буџет за ове скупље тикве!

🏆 Браво! Направили сте два модела регресије у једном часу. У завршном делу о регресији, научићете о логистичкој регресији за одређивање категорија.

## **🚀Изазов**

Тестирајте неколико различитих променљивих у овом нотебуку да видите како корелација утиче на тачност модела.

## [**Квиз након предавања**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Преглед и Самостално учење**

У овом часу смо научили о Линеарној регресији. Постоје и други важни типови регресије. Прочитајте о техникама Stepwise, Ridge, Lasso и Elasticnet. Добар курс за даље учење је [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Ако желите да научите више о коришћењу невероватног Tidymodels оквира, погледајте следеће ресурсе:

-   Веб-сајт Tidymodels: [Почните са Tidymodels](https://www.tidymodels.org/start/)

-   Мак Кун и Џулија Силџ, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **ХВАЛА:**

[Елисон Хорст](https://twitter.com/allison_horst?lang=en) за креирање невероватних илустрација које чине R приступачнијим и занимљивијим. Пронађите више илустрација у њеној [галерији](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако тежимо тачности, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква погрешна тумачења или неспоразуме који могу произаћи из коришћења овог превода.
